In [77]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('darkgrid')

import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from sklearn.feature_extraction.text import TfidfTransformer

from keras.preprocessing import text,sequence
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.util import ngrams
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Embedding
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import pickle



In [24]:

nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tristan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/tristan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
data = pd.DataFrame()
datadir = '../data/twittertf'
realorfake = {"True":1,"Fake":0}

for dir in os.listdir(datadir):
    temp_data = pd.read_csv(datadir + "/" +  dir)
    temp_data = temp_data.assign(real = realorfake.get(dir.split(".")[0]) )
    data = data.append(temp_data)

In [29]:
data.sample(5)

,title,text,subject,date,real
5105,Senate approves measure to kill Obama-era cont...,(Reuters) - Republicans in the U.S. Senate pas...,politicsNews,"March 6, 2017",1
13810,Bali airport's closure due to volcano has affe...,JAKARTA (Reuters) - The closure of I Gusti Ngu...,worldnews,"November 27, 2017",1
9733,DRAMA QUEEN CHRIS MATTHEWS Claims Republican ‘...,MSNBC host Chris Matthews on Thursday accused ...,politics,"Oct 7, 2017",0
11218,"Factbox: The economy, guns top on social media...",(Reuters) - The sixth televised Republican pre...,politicsNews,"January 15, 2016",1
1913,Trump Laughably Claims His Fake Obama Wiretap...,Remember when the world laughed at Donald Trum...,News,"April 3, 2017",0


In [30]:
stop_words=set(stopwords.words('english'))
punctuation=list(string.punctuation)
stop_words.update(punctuation)

In [37]:
def string_html(text):
    soup=BeautifulSoup(text,"html.parser")
    return soup.get_text()

def remove_square_brackets(text):
    return re.sub('\[[^]]*\]','',text)

def remove_URL(text):
    return re.sub(r'http\S+','',text)

def remove_stopwords(text):
    final_text=[]
    for i in text.split():
        if i.strip().lower() not in stop_words:
            final_text.append(i.strip())
    return " ".join(final_text)

def remove_HashOrAT(text):
    return re.sub('@|#','',text)

def remove_puncuation(text):
    return text.translate(str.maketrans('','',string.punctuation))

def remove_uppercase(text):
    return text.lower()
    

def clean_text_data(text):
    text=string_html(text)
    text=remove_square_brackets(text)
    text=remove_URL(text)
    text=remove_HashOrAT(text)
    text=remove_puncuation(text)
    text=remove_uppercase(text)
    return text

In [38]:

#data['text']=data['text'].apply(clean_text_data)

In [41]:
X_train,X_test,y_train,y_test=train_test_split(data.text,data.real,random_state=0)

In [57]:
text_clf = Pipeline(
    [('vec', CountVectorizer(preprocessor=clean_text_data, ngram_range=(1,3))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
    ])


In [60]:
text_clf = text_clf.fit(X_train, y_train)

In [61]:
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.9694432071269488

In [76]:
text_clf.predict(["Donald Trump is President of the United States"])

array([1])

In [78]:
with open("twitterTFModel.pkl",'wb') as file:
    pickle.dump(text_clf, file)
    

In [81]:
class model1_tweets:
    def __init__(self):
        self.model = pickle.load(open("twitterTFModel.pkl",'rb'))
        self.key = realorfake = {"True":1,"False":0}
        
    
    def preditWithText(self, text):
        return self.model.predict([text])[0]

    def return_str(self, text):
        if self.preditWithText(text) == 0:
            return False
        else:
            return True
        

In [82]:
newText = model1_tweets()
newText.preditWithText("Donald Trump is President")

0